In [1]:
options = ["piedra","tijeras","papel"]

In [3]:
search_winner("papel","tijeras")

2

In [8]:
def search_winner(p1,p2):
    if p1 == p2:
        result = 0
    
    elif p1 == "piedra" and p2 == "tijeras":
        result = 1
    elif p1 == "piedra" and p2 == "papel":
        result = 2
    elif p1 == "tijeras" and p2 == "piedra":
        result = 2
    elif p1 == "tijeras" and p2 == "papel":
        result = 1
    elif p1 == "papel" and p2 == "piedra":
        result = 1
    elif p1 == "papel" and p2 == "tijeras":
        result = 2
    return result


In [9]:
test = [
    ["piedra","piedra",0],
    ["piedra","tijeras",1],
    ["piedra","papel",2]
    
]

for partida in test:
    print("player 1: %s player2 : %s winner : %s validation: %s  " % (
        partida[0], partida[1], search_winner(partida[0], partida[1]), partida[2]))
    


player 1: piedra player2 : piedra winner : 0 validation: 0  
player 1: piedra player2 : tijeras winner : 1 validation: 1  
player 1: piedra player2 : papel winner : 2 validation: 2  


In [34]:
from random import choice
def get_choice():
    return choice(options)

In [35]:
for i in range(10):
    player1 = get_choice()
    player2 = get_choice()
    print("player 1: %s player2 : %s winner : %s " % (
        player1, player2, search_winner(player1, player2)))

player 1: piedra player2 : tijeras winner : 1 
player 1: tijeras player2 : tijeras winner : 0 
player 1: tijeras player2 : papel winner : 1 
player 1: tijeras player2 : papel winner : 1 
player 1: piedra player2 : piedra winner : 0 
player 1: piedra player2 : piedra winner : 0 
player 1: tijeras player2 : tijeras winner : 0 
player 1: piedra player2 : piedra winner : 0 
player 1: tijeras player2 : papel winner : 1 
player 1: papel player2 : tijeras winner : 2 


In [36]:
def str_to_list(option):
    if option == "piedra":
        return [1,0,0]
    elif option == "tijeras":
        return [0,1,0]
    else:
        return [0,0,1]

data_x = list(map(str_to_list, ["piedra","tijeras","papel"]))
data_y = list(map(str_to_list, ["papel","piedra","tijeras"]))

print(data_x)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [37]:
from sklearn.neural_network import MLPClassifier

In [38]:
clf = MLPClassifier(verbose=False, warm_start=True)

In [40]:
model = clf.fit([data_x[0]], [data_y[0]])
print(model)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=True)


In [41]:


def play_and_learn(iters=10, debug=False):
    score = {"win": 0, "loose": 0}
    
    data_x = []
    data_y = []
    
    for i in range(iters):
        player1 = get_choice()
        
        predict = model.predict_proba([str_to_list(player1)])[0]
        
        if predict[0] >= 0.99:
            player2 = options[0]
        elif predict[1] >= 0.99:
            player2 = options[1]
        elif predict[2] >= 0.99:
            player2 = options[2]
        else:
            player2 = get_choice()
            
        if debug==True:
            print("Player1: %s Player2 (modelo): %s --> %s" % (player1, predict, player2))
        
        winner = search_winner(player1, player2)
        if debug==True:
            print("Comprobamos: p1 VS p2: %s" % winner)
        
        if winner==2:
            data_x.append(str_to_list(player1))
            data_y.append(str_to_list(player2))
            
            score["win"]+=1
        else:
            score["loose"]+=1
        
    return score, data_x, data_y



In [42]:
score, data_x, data_y = play_and_learn(1, debug=True)  
print(data_x)
print(data_y)
print("Score: %s %s %%" % (score, (score["win"]*100/(score["win"]+score["loose"]))))
if len(data_x):
    model = model.partial_fit(data_x, data_y)

Player1: tijeras Player2 (modelo): [0.11740669 0.09208506 0.90562861] --> tijeras
Comprobamos: p1 VS p2: 0
[]
[]
Score: {'win': 0, 'loose': 1} 0.0 %


In [43]:
i = 0
historic_pct = []
while True:
    i+=1
    score,data_x,data_y = play_and_learn(1000,debug=False)
    porcent = (score["win"]*100/(score["win"]+score["loose"]))
    historic_pct.append(porcent)
    print("Iter: %s - score: %s %s %%" % (i, score, porcent))
    if len(data_x):
        model = model.partial_fit(data_x, data_y)
        
    if sum(historic_pct[-9:])==900: # si lo haz hecho bien al 100% 9 veces, entonces para
        break
    

Iter: 1 - score: {'win': 356, 'loose': 644} 35.6 %
Iter: 2 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 3 - score: {'win': 324, 'loose': 676} 32.4 %
Iter: 4 - score: {'win': 337, 'loose': 663} 33.7 %
Iter: 5 - score: {'win': 330, 'loose': 670} 33.0 %
Iter: 6 - score: {'win': 314, 'loose': 686} 31.4 %
Iter: 7 - score: {'win': 371, 'loose': 629} 37.1 %
Iter: 8 - score: {'win': 334, 'loose': 666} 33.4 %
Iter: 9 - score: {'win': 331, 'loose': 669} 33.1 %
Iter: 10 - score: {'win': 349, 'loose': 651} 34.9 %
Iter: 11 - score: {'win': 355, 'loose': 645} 35.5 %
Iter: 12 - score: {'win': 319, 'loose': 681} 31.9 %
Iter: 13 - score: {'win': 327, 'loose': 673} 32.7 %
Iter: 14 - score: {'win': 338, 'loose': 662} 33.8 %
Iter: 15 - score: {'win': 330, 'loose': 670} 33.0 %
Iter: 16 - score: {'win': 345, 'loose': 655} 34.5 %
Iter: 17 - score: {'win': 331, 'loose': 669} 33.1 %
Iter: 18 - score: {'win': 323, 'loose': 677} 32.3 %
Iter: 19 - score: {'win': 345, 'loose': 655} 34.5 %
Iter: 20 - score: {'w

Iter: 158 - score: {'win': 342, 'loose': 658} 34.2 %
Iter: 159 - score: {'win': 335, 'loose': 665} 33.5 %
Iter: 160 - score: {'win': 330, 'loose': 670} 33.0 %
Iter: 161 - score: {'win': 321, 'loose': 679} 32.1 %
Iter: 162 - score: {'win': 335, 'loose': 665} 33.5 %
Iter: 163 - score: {'win': 313, 'loose': 687} 31.3 %
Iter: 164 - score: {'win': 343, 'loose': 657} 34.3 %
Iter: 165 - score: {'win': 341, 'loose': 659} 34.1 %
Iter: 166 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 167 - score: {'win': 333, 'loose': 667} 33.3 %
Iter: 168 - score: {'win': 332, 'loose': 668} 33.2 %
Iter: 169 - score: {'win': 347, 'loose': 653} 34.7 %
Iter: 170 - score: {'win': 357, 'loose': 643} 35.7 %
Iter: 171 - score: {'win': 345, 'loose': 655} 34.5 %
Iter: 172 - score: {'win': 338, 'loose': 662} 33.8 %
Iter: 173 - score: {'win': 335, 'loose': 665} 33.5 %
Iter: 174 - score: {'win': 320, 'loose': 680} 32.0 %
Iter: 175 - score: {'win': 324, 'loose': 676} 32.4 %
Iter: 176 - score: {'win': 337, 'loose': 663} 

Iter: 313 - score: {'win': 326, 'loose': 674} 32.6 %
Iter: 314 - score: {'win': 313, 'loose': 687} 31.3 %
Iter: 315 - score: {'win': 347, 'loose': 653} 34.7 %
Iter: 316 - score: {'win': 322, 'loose': 678} 32.2 %
Iter: 317 - score: {'win': 314, 'loose': 686} 31.4 %
Iter: 318 - score: {'win': 334, 'loose': 666} 33.4 %
Iter: 319 - score: {'win': 332, 'loose': 668} 33.2 %
Iter: 320 - score: {'win': 303, 'loose': 697} 30.3 %
Iter: 321 - score: {'win': 322, 'loose': 678} 32.2 %
Iter: 322 - score: {'win': 342, 'loose': 658} 34.2 %
Iter: 323 - score: {'win': 343, 'loose': 657} 34.3 %
Iter: 324 - score: {'win': 324, 'loose': 676} 32.4 %
Iter: 325 - score: {'win': 298, 'loose': 702} 29.8 %
Iter: 326 - score: {'win': 333, 'loose': 667} 33.3 %
Iter: 327 - score: {'win': 334, 'loose': 666} 33.4 %
Iter: 328 - score: {'win': 350, 'loose': 650} 35.0 %
Iter: 329 - score: {'win': 318, 'loose': 682} 31.8 %
Iter: 330 - score: {'win': 326, 'loose': 674} 32.6 %
Iter: 331 - score: {'win': 547, 'loose': 453} 

In [50]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show,output_notebook
output_notebook()


Loading BokehJS ...

In [49]:
x = range(len(historic_pct))
y = historic_pct
p = figure(
    title = "% de aprendizaje en cada iteración",
          x_axis_label="Iter",y_axis_label="%", width = 900,
        )
p.line(x,y,legend=None, line_width=1)
show(p)